In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import datetime
import calendar
from scipy import sparse
from pandas.api.types import CategoricalDtype
import time
from tqdm import tqdm

In [ ]:
cd D:\Course\SBRS

In [ ]:
# ctg_file = 'datasets/retailrocket/category_tree.csv'
events_file = 'datasets/retailrocket/events.csv'
# item_prop_1 = 'datasets/retailrocket/item_properties_part1.csv'
# item_prop_2 = 'datasets/retailrocket/item_properties_part2.csv'

In [ ]:
# ctg_df = pd.read_csv(ctg_file)
events_df = pd.read_csv(events_file)
# item_prop_1_df = pd.read_csv(item_prop_1)
# item_prop_2_df = pd.read_csv(item_prop_2)


In [ ]:
events_df.head()

In [ ]:
events_actions = {
    'view': 1,
    'addtocart':2,
    'transaction':3
}

In [ ]:
events_df.event = [events_actions[item] for item in events_df.event]

In [ ]:
events_df.head()

In [ ]:
events_df['event_date'] = pd.to_datetime(events_df['timestamp'])
events_df = events_df.drop('timestamp', axis=1)


In [ ]:
events_df['date'] = events_df.event_date.dt.date
events_df['time'] = events_df.event_date.dt.time
events_df['year'] = events_df.event_date.dt.year
events_df['month'] = events_df.event_date.dt.month
events_df['day'] = events_df.event_date.dt.day
events_df['hour'] = events_df.event_date.dt.hour
events_df['minute'] = events_df.event_date.dt.minute
events_df['sec'] = events_df.event_date.dt.second
events_df = events_df.drop(['event_date','time','date'], axis=1)

In [ ]:
events_df.head(1000000)

In [ ]:
new = events_df.sort_values('visitorid')
new.head(10000)

In [ ]:
list_df = []
for i in tqdm(range(len(events_df.visitorid.unique()))):
    tqdm.sleep(1)
    list_df.append(events_df[events_df.visitorid == i].itemid.to_list())
list_df

In [ ]:
events_df.transactionid.fillna(-1, inplace=True)

In [ ]:
events_df.head()

In [ ]:
users = events_df.visitorid.unique()
items = events_df.itemid.unique()
shape = (len(users),len(items))

In [ ]:
# Categorical coding
users_cat = CategoricalDtype(categories=sorted(users), ordered=True)
items_cat = CategoricalDtype(categories=sorted(items), ordered=True)
users_index = events_df.visitorid.astype(users_cat).cat.codes
items_index = events_df.itemid.astype(items_cat).cat.codes

In [ ]:
# To csr
coo = sparse.coo_matrix((events_df.event, (users_index, items_index)), shape=shape)
csr = coo.tocsr()

In [ ]:
# Все записи (относящиеся к одному пользоветелю), временной промежуток между которыми менее 10 минут, - сессия 
